In [1]:
import pickle
import numpy as np
from scipy.special import softmax

rng = np.random.default_rng()

In [2]:
with open("data/train_data.pkl", "rb") as train_file:
    train_data = pickle.load(train_file)

with open("data/test_data.pkl", "rb") as test_file:
    test_data = pickle.load(test_file)

In [3]:
# Define the network

img_px = 28
n_i = 512
n_1 = 128
n_2 = 64
n_L = 10 # there are 10 classes in FashionMNIST

k_i = 1/np.sqrt( img_px * img_px )
k_1 = 1/np.sqrt( n_i * n_i )
k_2 = 1/np.sqrt( n_1 * n_1)
k_L = 1/np.sqrt( n_2 * n_2 )

w_i = rng.uniform( -k_i, k_i, (n_i, img_px*img_px) )
b_i = rng.normal( -k_i, k_i, size=n_i )

w_1 = rng.normal( -k_1, k_1, size=(n_1, n_i) )
b_1 = rng.normal( -k_1, k_1, size=n_1 )

w_2 = rng.normal( -k_2, k_2, size=(n_2, n_1))
b_2 = rng.normal( -k_2, k_2, size=n_2 )

w_L = rng.normal( -k_L, k_L, size=(n_L, n_2) )
b_L = rng.normal( -k_L, k_L, size=n_L )

In [4]:
# forward function, used to make predictions

def forward(x, return_z_a=False):
    x = x.flatten()
    
    z_i = w_i @ x + b_i
    a_i = np.maximum(0, z_i)

    z_1 = w_1 @ a_i + b_1
    a_1 = np.maximum(0, z_1)

    z_2 = w_2 @ a_1 + b_2
    a_2 = np.maximum(0, z_2)

    z_L = w_L @ a_2 + b_L
    a_L = softmax(z_L)

    if return_z_a:
        return (z_i, z_1, z_2, z_L), (a_i, a_1, a_2, a_L)
    else:
        return a_L

forward(train_data[0][0])

array([0.1004151 , 0.09923117, 0.10055567, 0.10118387, 0.09933581,
       0.10178994, 0.09841747, 0.0987981 , 0.10017248, 0.10010039])

In [5]:
# Cross-entropy loss

def loss_fn(y, a_L):
    return - (y * np.log(a_L)).sum()   


def test():
    n_samples = len(test_data)
    correct = 0
    sum_loss = 0
    for x, y in test_data:
        y_hat = forward(x)
        correct += y.argmax() == y_hat.argmax()
        sum_loss += loss_fn(y, y_hat)
    return correct / n_samples, sum_loss / n_samples        

test()

(0.1, 2.302635596332296)

In [6]:
def ReLU_derivative(x):
    r = np.ones(x.shape)
    r[x == 0] = 0
    return r

def calc_derivatives(x, y):
    x = x.flatten()
    
    (z_i, z_1, z_2, z_L), (a_i, a_1, a_2, a_L) = forward(x, True)
    
    nabla_z_L = a_L - y
    nabla_w_L = np.outer(nabla_z_L, a_2)
    
    nabla_z_2 = ReLU_derivative(z_2) * (w_L.T @ nabla_z_L)
    nabla_w_2 = np.outer(nabla_z_2, a_1)
    
    nabla_z_1 = ReLU_derivative(z_1) * (w_2.T @ nabla_z_2)
    nabla_w_1 = np.outer(nabla_z_1, a_i)
    
    nabla_z_i = ReLU_derivative(z_i) * (w_1.T @ nabla_z_1)
    nabla_w_i = np.outer(nabla_z_i, x)

    return (nabla_w_i, nabla_w_1, nabla_w_2, nabla_w_L), (nabla_z_i, nabla_z_1, nabla_z_2, nabla_z_L), loss_fn(y, a_L)

In [7]:
# train loop

learning_rate = 1e-3
mini_batch_size = 64
training_epochs = 5

n_mini_batches = len(train_data) // mini_batch_size
training_index = np.arange(len(train_data))

print("Pre-train stats")
accuracy, avg_loss = test()
print(f"==> Accuracy: {100*accuracy:0.1f}%, Avg loss: {avg_loss:>8f}\n")

for epoch in range(training_epochs):
    print(f"Epoch {epoch}\n" + 20*'-')
    rng.shuffle(training_index)
    for mini_batch_number in range(n_mini_batches):
        # batch average derivatives
        sum_nabla_w_i = np.zeros(w_i.shape)
        sum_nabla_w_1 = np.zeros(w_1.shape)
        sum_nabla_w_2 = np.zeros(w_2.shape)
        sum_nabla_w_L = np.zeros(w_L.shape)
        sum_nabla_b_i = np.zeros(b_i.shape)
        sum_nabla_b_1 = np.zeros(b_1.shape)
        sum_nabla_b_2 = np.zeros(b_2.shape)
        sum_nabla_b_L = np.zeros(b_L.shape)
        sum_loss = 0
        mini_batch = train_data[mini_batch_number*mini_batch_size : (mini_batch_number+1)*mini_batch_size]
        for x, y in mini_batch:
            (nabla_w_i, nabla_w_1, nabla_w_2, nabla_w_L), (nabla_b_i, nabla_b_1, nabla_b_2, nabla_b_L), loss = calc_derivatives(x, y)
            sum_nabla_w_i += nabla_w_i
            sum_nabla_w_1 += nabla_w_1
            sum_nabla_w_2 += nabla_w_2
            sum_nabla_w_L += nabla_w_L
            sum_nabla_b_i += nabla_b_i
            sum_nabla_b_1 += nabla_b_1
            sum_nabla_b_2 += nabla_b_2
            sum_nabla_b_L += nabla_b_L
            sum_loss += loss
        w_i -= learning_rate * (sum_nabla_w_i / len(mini_batch))
        w_1 -= learning_rate * (sum_nabla_w_1 / len(mini_batch))
        w_2 -= learning_rate * (sum_nabla_w_2 / len(mini_batch))
        w_L -= learning_rate * (sum_nabla_w_L / len(mini_batch))
        b_i -= learning_rate * (sum_nabla_b_i / len(mini_batch))
        b_1 -= learning_rate * (sum_nabla_b_1 / len(mini_batch))
        b_2 -= learning_rate * (sum_nabla_b_2 / len(mini_batch))
        b_L -= learning_rate * (sum_nabla_b_L / len(mini_batch))
        if mini_batch_number % 100 == 0:
            print(f"loss: {sum_loss/len(mini_batch):>8f} [mini-batch {mini_batch_number} / {n_mini_batches}]")
    # Test at the end of each epoch
    accuracy, avg_loss = test()
    print(f"==> Accuracy: {100*accuracy:0.1f}%, Avg loss: {avg_loss:>8f}\n")

Pre-train stats
==> Accuracy: 10.0%, Avg loss: 2.302636

Epoch 0
--------------------
loss: 2.300198 [mini-batch 0 / 937]
loss: 2.301184 [mini-batch 100 / 937]
loss: 2.301416 [mini-batch 200 / 937]
loss: 2.257401 [mini-batch 300 / 937]
loss: 1.650731 [mini-batch 400 / 937]
loss: 1.316656 [mini-batch 500 / 937]
loss: 1.239089 [mini-batch 600 / 937]
loss: 1.042560 [mini-batch 700 / 937]
loss: 1.082339 [mini-batch 800 / 937]
loss: 0.960052 [mini-batch 900 / 937]
==> Accuracy: 59.2%, Avg loss: 0.970282

Epoch 1
--------------------
loss: 0.985221 [mini-batch 0 / 937]
loss: 0.991065 [mini-batch 100 / 937]
loss: 0.754201 [mini-batch 200 / 937]
loss: 0.936900 [mini-batch 300 / 937]
loss: 0.799626 [mini-batch 400 / 937]
loss: 0.675703 [mini-batch 500 / 937]
loss: 0.836319 [mini-batch 600 / 937]
loss: 0.749523 [mini-batch 700 / 937]
loss: 0.807046 [mini-batch 800 / 937]
loss: 0.722393 [mini-batch 900 / 937]
==> Accuracy: 72.7%, Avg loss: 0.716839

Epoch 2
--------------------
loss: 0.633655 [mi

/tmp/ipykernel_22102/2596950636.py:4: RuntimeWarning: divide by zero encountered in log
  return - (y * np.log(a_L)).sum()
/tmp/ipykernel_22102/2596950636.py:4: RuntimeWarning: invalid value encountered in multiply
  return - (y * np.log(a_L)).sum()
/tmp/ipykernel_22102/1772072483.py:9: RuntimeWarning: overflow encountered in matmul
  z_1 = w_1 @ a_i + b_1
/tmp/ipykernel_22102/1772072483.py:12: RuntimeWarning: invalid value encountered in matmul
  z_2 = w_2 @ a_1 + b_2


loss:      nan [mini-batch 300 / 937]
loss:      nan [mini-batch 400 / 937]
loss:      nan [mini-batch 500 / 937]
loss:      nan [mini-batch 600 / 937]
loss:      nan [mini-batch 700 / 937]
loss:      nan [mini-batch 800 / 937]
loss:      nan [mini-batch 900 / 937]
==> Accuracy: 10.0%, Avg loss:      nan

Epoch 12
--------------------
loss:      nan [mini-batch 0 / 937]
loss:      nan [mini-batch 100 / 937]
loss:      nan [mini-batch 200 / 937]
loss:      nan [mini-batch 300 / 937]
loss:      nan [mini-batch 400 / 937]
loss:      nan [mini-batch 500 / 937]
loss:      nan [mini-batch 600 / 937]
loss:      nan [mini-batch 700 / 937]
loss:      nan [mini-batch 800 / 937]
loss:      nan [mini-batch 900 / 937]
==> Accuracy: 10.0%, Avg loss:      nan

Epoch 13
--------------------
loss:      nan [mini-batch 0 / 937]
loss:      nan [mini-batch 100 / 937]
loss:      nan [mini-batch 200 / 937]
loss:      nan [mini-batch 300 / 937]
loss:      nan [mini-batch 400 / 937]
loss:      nan [mini-batch